In [7]:
# 初始化环境
import os
from os.path import *
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
file_dir = os.path.abspath(".")
proj_dir = dirname(dirname(file_dir))
os.environ['HADOOP_HOME'] = r"C:\hadoop\hadoop-2.6.5"
os.environ['SPARK_HOME'] = r"C:\hadoop\spark-2.4.5-bin-hadoop2.6"
os.environ['JAVA_HOME'] = r"C:\Program Files\Java\jdk1.8.0_191"
os.environ["SCALA_HOME"]=r"C:\Program Files (x86)\scala"
sconf = SparkConf().setMaster("local[4]").setAppName("test")
sc = SparkContext(conf=sconf)
ss = SparkSession(sparkContext=sc)
ss

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test, master=local[4]) created by __init__ at <ipython-input-1-46b48f294111>:14 

In [155]:
# 按照频率构造测试数据
test_dict = [chr(i)+chr(i)+chr(i) for i in range(97, 97+26)]
# 创建供采样的数组，里面的字符按顺序添加，令z->x逐渐减少
test_dict_to_sample = []
for c,i in zip(test_dict, range(1,100)):
    for _ in range(i):
        test_dict_to_sample.append(c)
print(len(test_dict_to_sample))
import random
#random.seed(1234)
def gen_test_x(N=100,k=10):
    test_x_gen = []
    for i in range(N):
        sample_data = " ".join(random.sample(test_dict_to_sample, k=k))
        test_x_gen.append(sample_data)
    return test_x_gen
test_x = gen_test_x(100)
",".join(test_dict),test_x

351


('aaa,bbb,ccc,ddd,eee,fff,ggg,hhh,iii,jjj,kkk,lll,mmm,nnn,ooo,ppp,qqq,rrr,sss,ttt,uuu,vvv,www,xxx,yyy,zzz',
 ['vvv nnn eee ttt hhh mmm uuu zzz xxx kkk',
  'qqq www yyy nnn mmm sss vvv sss kkk www',
  'xxx yyy yyy rrr uuu nnn nnn rrr yyy zzz',
  'zzz ggg kkk iii uuu www rrr iii rrr iii',
  'qqq zzz zzz iii jjj ttt rrr xxx mmm zzz',
  'sss ooo rrr yyy ppp hhh qqq xxx www www',
  'ppp kkk yyy yyy zzz zzz sss qqq zzz yyy',
  'xxx nnn uuu vvv ppp iii qqq www www vvv',
  'lll rrr ppp uuu mmm iii ppp vvv fff zzz',
  'jjj nnn zzz sss qqq xxx vvv eee zzz ppp',
  'mmm qqq ooo ooo rrr jjj zzz ppp rrr xxx',
  'vvv xxx www rrr yyy rrr zzz lll www rrr',
  'lll vvv ppp yyy lll sss vvv uuu hhh eee',
  'www lll ddd ggg qqq rrr xxx hhh www fff',
  'ppp ppp ttt yyy zzz sss vvv rrr rrr uuu',
  'bbb yyy lll ppp lll ppp vvv yyy uuu vvv',
  'ttt nnn ppp vvv uuu kkk jjj hhh vvv yyy',
  'vvv zzz xxx uuu hhh nnn nnn yyy vvv yyy',
  'hhh qqq ggg eee ttt ttt zzz fff ooo qqq',
  'jjj yyy zzz yyy ppp rrr ooo uuu yy

In [22]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
x_train = [
    "1 1 1",
    "2 2 2",
    "2 2 2",
    "3 3 3"
]
y_train = [0, 0, 0, 1]
spark_input_data =  [i for i in zip(x_train, y_train)]
spark_input_data

[('1 1 1', 0), ('2 2 2', 0), ('2 2 2', 0), ('3 3 3', 1)]

In [144]:
test_x_spark = [(line, 1) for line in test_x]
sentenceData = ss.createDataFrame(test_x_spark, ["x","y"])
tokenizer = Tokenizer(inputCol="x", outputCol="x_toks")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+--------------------+---+--------------------+
|                   x|  y|              x_toks|
+--------------------+---+--------------------+
|ppp ppp qqq yyy m...|  1|[ppp, ppp, qqq, y...|
|nnn yyy iii www w...|  1|[nnn, yyy, iii, w...|
|xxx qqq zzz nnn w...|  1|[xxx, qqq, zzz, n...|
|xxx zzz uuu xxx u...|  1|[xxx, zzz, uuu, x...|
|lll yyy mmm ttt u...|  1|[lll, yyy, mmm, t...|
|rrr yyy ppp ttt p...|  1|[rrr, yyy, ppp, t...|
|ooo www ttt zzz q...|  1|[ooo, www, ttt, z...|
|ooo www kkk yyy w...|  1|[ooo, www, kkk, y...|
|lll rrr zzz vvv m...|  1|[lll, rrr, zzz, v...|
|zzz lll ttt ooo u...|  1|[zzz, lll, ttt, o...|
|xxx ooo sss vvv z...|  1|[xxx, ooo, sss, v...|
|hhh kkk nnn rrr r...|  1|[hhh, kkk, nnn, r...|
|yyy lll zzz iii g...|  1|[yyy, lll, zzz, i...|
|www iii ooo ttt u...|  1|[www, iii, ooo, t...|
|qqq mmm hhh ttt y...|  1|[qqq, mmm, hhh, t...|
|lll jjj www sss s...|  1|[lll, jjj, www, s...|
|www vvv lll www j...|  1|[www, vvv, lll, w...|
|lll rrr www mmm y...|  1|[lll, rrr, www

In [145]:
# HashingTF是按词频进行Hashing
hashingTF = HashingTF(inputCol="x_toks",
                      outputCol="x_hashing_tf",
                      numFeatures=3,
                      #binary=True
                      )
featurizedData = hashingTF.transform(wordsData)
featurizedData.show()

+--------------------+---+--------------------+--------------------+
|                   x|  y|              x_toks|        x_hashing_tf|
+--------------------+---+--------------------+--------------------+
|ppp ppp qqq yyy m...|  1|[ppp, ppp, qqq, y...| (3,[0,1],[8.0,2.0])|
|nnn yyy iii www w...|  1|[nnn, yyy, iii, w...|(3,[0,1,2],[7.0,1...|
|xxx qqq zzz nnn w...|  1|[xxx, qqq, zzz, n...|(3,[0,1,2],[3.0,5...|
|xxx zzz uuu xxx u...|  1|[xxx, zzz, uuu, x...|(3,[0,1,2],[1.0,4...|
|lll yyy mmm ttt u...|  1|[lll, yyy, mmm, t...|(3,[0,1,2],[5.0,4...|
|rrr yyy ppp ttt p...|  1|[rrr, yyy, ppp, t...|(3,[0,1,2],[5.0,3...|
|ooo www ttt zzz q...|  1|[ooo, www, ttt, z...|(3,[0,1,2],[5.0,3...|
|ooo www kkk yyy w...|  1|[ooo, www, kkk, y...|(3,[0,1,2],[4.0,2...|
|lll rrr zzz vvv m...|  1|[lll, rrr, zzz, v...|(3,[0,1,2],[7.0,2...|
|zzz lll ttt ooo u...|  1|[zzz, lll, ttt, o...|(3,[0,1,2],[3.0,5...|
|xxx ooo sss vvv z...|  1|[xxx, ooo, sss, v...|(3,[0,1,2],[4.0,4...|
|hhh kkk nnn rrr r...|  1|[hhh, kk

In [146]:
# 显示密向量
[ v[0].toArray() for v in featurizedData.select("x_hashing_tf").rdd.collect() ]

[array([8., 2., 0.]),
 array([7., 1., 2.]),
 array([3., 5., 2.]),
 array([1., 4., 5.]),
 array([5., 4., 1.]),
 array([5., 3., 2.]),
 array([5., 3., 2.]),
 array([4., 2., 4.]),
 array([7., 2., 1.]),
 array([3., 5., 2.]),
 array([4., 4., 2.]),
 array([5., 3., 2.]),
 array([6., 2., 2.]),
 array([5., 1., 4.]),
 array([6., 3., 1.]),
 array([3., 5., 2.]),
 array([5., 2., 3.]),
 array([5., 2., 3.]),
 array([4., 3., 3.]),
 array([5., 2., 3.]),
 array([6., 2., 2.]),
 array([5., 3., 2.]),
 array([7., 2., 1.]),
 array([5., 3., 2.]),
 array([2., 5., 3.]),
 array([2., 5., 3.]),
 array([3., 4., 3.]),
 array([4., 3., 3.]),
 array([3., 3., 4.]),
 array([3., 3., 4.]),
 array([3., 3., 4.]),
 array([7., 3., 0.]),
 array([4., 3., 3.]),
 array([4., 2., 4.]),
 array([3., 2., 5.]),
 array([5., 0., 5.]),
 array([3., 2., 5.]),
 array([3., 4., 3.]),
 array([4., 3., 3.]),
 array([2., 5., 3.]),
 array([7., 2., 1.]),
 array([3., 4., 3.]),
 array([4., 3., 3.]),
 array([4., 4., 2.]),
 array([3., 4., 3.]),
 array([5.

In [147]:
idf = IDF(inputCol="x_hashing_tf", outputCol="x_idf")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.show()

+--------------------+---+--------------------+--------------------+--------------------+
|                   x|  y|              x_toks|        x_hashing_tf|               x_idf|
+--------------------+---+--------------------+--------------------+--------------------+
|ppp ppp qqq yyy m...|  1|[ppp, ppp, qqq, y...| (3,[0,1],[8.0,2.0])|(3,[0,1],[0.0,0.0...|
|nnn yyy iii www w...|  1|[nnn, yyy, iii, w...|(3,[0,1,2],[7.0,1...|(3,[0,1,2],[0.0,0...|
|xxx qqq zzz nnn w...|  1|[xxx, qqq, zzz, n...|(3,[0,1,2],[3.0,5...|(3,[0,1,2],[0.0,0...|
|xxx zzz uuu xxx u...|  1|[xxx, zzz, uuu, x...|(3,[0,1,2],[1.0,4...|(3,[0,1,2],[0.0,0...|
|lll yyy mmm ttt u...|  1|[lll, yyy, mmm, t...|(3,[0,1,2],[5.0,4...|(3,[0,1,2],[0.0,0...|
|rrr yyy ppp ttt p...|  1|[rrr, yyy, ppp, t...|(3,[0,1,2],[5.0,3...|(3,[0,1,2],[0.0,0...|
|ooo www ttt zzz q...|  1|[ooo, www, ttt, z...|(3,[0,1,2],[5.0,3...|(3,[0,1,2],[0.0,0...|
|ooo www kkk yyy w...|  1|[ooo, www, kkk, y...|(3,[0,1,2],[4.0,2...|(3,[0,1,2],[0.0,0...|
|lll rrr z

In [148]:
[ v[0].toArray() for v in rescaledData.select("x_idf").rdd.collect() ]

[array([0.        , 0.04000133, 0.        ]),
 array([0.        , 0.02000067, 0.06030608]),
 array([0.        , 0.10000333, 0.06030608]),
 array([0.        , 0.08000267, 0.15076519]),
 array([0.        , 0.08000267, 0.03015304]),
 array([0.        , 0.060002  , 0.06030608]),
 array([0.        , 0.060002  , 0.06030608]),
 array([0.        , 0.04000133, 0.12061215]),
 array([0.        , 0.04000133, 0.03015304]),
 array([0.        , 0.10000333, 0.06030608]),
 array([0.        , 0.08000267, 0.06030608]),
 array([0.        , 0.060002  , 0.06030608]),
 array([0.        , 0.04000133, 0.06030608]),
 array([0.        , 0.02000067, 0.12061215]),
 array([0.        , 0.060002  , 0.03015304]),
 array([0.        , 0.10000333, 0.06030608]),
 array([0.        , 0.04000133, 0.09045911]),
 array([0.        , 0.04000133, 0.09045911]),
 array([0.        , 0.060002  , 0.09045911]),
 array([0.        , 0.04000133, 0.09045911]),
 array([0.        , 0.04000133, 0.06030608]),
 array([0.        , 0.060002  , 0.

很奇怪，映射后第0维度全是0

In [162]:
sc.stop()

In [149]:
# 和sklearn的tfidf对比
import sklearn
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
sk_tf = TfidfVectorizer(use_idf=True,
                        smooth_idf=True,
                        norm=None,
                        stop_words=[])
x_tf = sk_tf.fit_transform(test_x)
pd.DataFrame(data = x_tf.toarray(),columns=sk_tf.get_feature_names())


,bbb,ccc,ddd,eee,fff,ggg,hhh,iii,jjj,kkk,...,qqq,rrr,sss,ttt,uuu,vvv,www,xxx,yyy,zzz
0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.247825,2.181133,2.149385,...,1.901548,0.000000,0.000000,0.000000,0.000000,1.572069,0.000000,0.000000,1.554678,0.000000
1,0.000000,0.00000,0.000000,0.000000,2.570598,0.000000,2.524078,2.247825,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.366590,0.000000,3.109355,1.589769
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.181133,2.149385,...,1.901548,0.000000,0.000000,0.000000,0.000000,1.572069,1.683295,1.644829,0.000000,4.769306
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.683295,3.707841,0.000000,1.683295,3.289657,0.000000,4.769306
4,0.000000,0.00000,0.000000,3.050171,0.000000,0.000000,0.000000,0.000000,4.362267,0.000000,...,0.000000,0.000000,0.000000,1.683295,1.853920,1.572069,0.000000,0.000000,1.554678,1.589769
5,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.181133,0.000000,...,0.000000,1.853920,2.118613,1.683295,1.853920,0.000000,0.000000,0.000000,1.554678,1.589769
6,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.901548,1.853920,2.118613,1.683295,0.000000,0.000000,1.683295,0.000000,1.554678,1.589769
7,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.149385,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.572069,5.049885,1.644829,3.109355,0.000000
8,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.362267,0.000000,...,0.000000,1.853920,0.000000,0.000000,0.000000,1.572069,0.000000,0.000000,0.000000,1.589769
9,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.149385,...,0.000000,1.853920,0.000000,1.683295,1.853920,3.144138,0.000000,0.000000,0.000000,1.589769


这个结果，也就是说a根本就没有被随机出来，越往后的出场率越高，对每个文章都是一样的
如果我们的数据很长的话呢？

In [159]:
test_x_2 = gen_test_x(N=100,k=len(test_dict_to_sample)-100) # 几乎使用全量数据来sample，这样得到的数据只是相当于乱序了
test_x_2

['vvv ppp nnn uuu www zzz yyy jjj ppp kkk lll iii nnn lll yyy lll sss sss ppp hhh ddd rrr rrr ggg www uuu ggg ooo nnn qqq qqq ccc yyy ttt sss qqq www ttt yyy xxx www zzz jjj ddd ooo uuu ppp xxx qqq ttt xxx uuu fff sss xxx ttt jjj hhh fff mmm sss qqq xxx rrr hhh iii www qqq yyy sss fff uuu ttt aaa kkk bbb jjj ppp vvv mmm www yyy uuu zzz rrr yyy yyy iii kkk mmm zzz xxx uuu ppp yyy ttt sss sss rrr uuu fff vvv www vvv sss jjj nnn www www mmm sss xxx zzz ooo ccc ooo www vvv vvv uuu xxx qqq www mmm yyy www qqq zzz jjj ooo yyy uuu sss uuu rrr zzz uuu xxx yyy qqq zzz yyy qqq ddd www rrr eee qqq rrr zzz kkk ttt vvv hhh nnn vvv xxx nnn ttt sss eee vvv ooo kkk ttt yyy www rrr www yyy ooo vvv ppp lll xxx sss ggg uuu ppp xxx vvv uuu ooo xxx ppp rrr ooo vvv rrr www ggg uuu xxx iii zzz rrr yyy hhh yyy ooo zzz ttt ttt ooo vvv mmm lll ooo uuu zzz nnn ppp nnn zzz mmm mmm lll yyy uuu www uuu qqq ttt lll jjj yyy zzz xxx xxx ttt rrr eee vvv nnn ppp xxx ttt vvv yyy ttt ggg zzz ggg vvv ttt lll hhh www rrr zz

In [160]:
x_tf = sk_tf.fit_transform(test_x_2)
pd.DataFrame(data = x_tf.toarray(),columns=sk_tf.get_feature_names())

,aaa,bbb,ccc,ddd,eee,fff,ggg,hhh,iii,jjj,...,qqq,rrr,sss,ttt,uuu,vvv,www,xxx,yyy,zzz
0,1.258412,1.093332,2.101545,3.029851,3.0,4.0,6.0,6.0,4.0,7.0,...,13.0,14.0,13.0,16.0,18.0,16.0,18.0,17.0,20.0,16.0
1,0.000000,0.000000,2.101545,3.029851,3.0,4.0,6.0,7.0,7.0,8.0,...,11.0,12.0,16.0,13.0,18.0,15.0,15.0,14.0,22.0,15.0
2,1.258412,2.186664,3.152317,3.029851,3.0,6.0,5.0,6.0,5.0,8.0,...,11.0,14.0,12.0,15.0,17.0,15.0,15.0,16.0,17.0,19.0
3,1.258412,1.093332,3.152317,4.039801,5.0,5.0,6.0,6.0,7.0,7.0,...,16.0,11.0,9.0,11.0,14.0,16.0,15.0,19.0,21.0,19.0
4,1.258412,0.000000,3.152317,3.029851,5.0,4.0,6.0,6.0,7.0,8.0,...,15.0,13.0,14.0,12.0,12.0,17.0,17.0,16.0,15.0,16.0
5,1.258412,2.186664,2.101545,2.019901,5.0,6.0,5.0,4.0,7.0,6.0,...,10.0,14.0,13.0,14.0,14.0,13.0,20.0,20.0,19.0,16.0
6,0.000000,1.093332,2.101545,2.019901,3.0,4.0,7.0,6.0,5.0,3.0,...,9.0,15.0,12.0,19.0,13.0,15.0,14.0,21.0,20.0,21.0
7,0.000000,2.186664,0.000000,4.039801,5.0,5.0,4.0,6.0,7.0,7.0,...,14.0,13.0,15.0,11.0,15.0,17.0,16.0,19.0,19.0,16.0
8,0.000000,2.186664,2.101545,3.029851,4.0,4.0,4.0,5.0,7.0,9.0,...,14.0,11.0,9.0,11.0,17.0,13.0,21.0,19.0,16.0,19.0
9,1.258412,1.093332,2.101545,4.039801,3.0,3.0,7.0,5.0,7.0,7.0,...,15.0,16.0,12.0,14.0,16.0,13.0,17.0,13.0,17.0,15.0


显然，这样得到的数据比例非常匀称

In [161]:
print("end")

end
